In [1]:
!pip install -q pyjwt

In [2]:
import requests, jwt
from IPython.display import JSON

CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/myrealm/protocol/openid-connect/token"

# Bootstraping Lakekeeper
This Notebook performs bootstrapping via python requests. It only works if the server hasn't previously bootstrapped using the UI!

## 1. Sign in
First, we need to obtain a token from our Identity Provider. In this example a `Keycloak` is running beside Lakekeeper. A few users have been pre-created in Keycloak for this example. We are now logging into Keycloak as the technical user (client) `trino`. If a human user bootstraps the catalog, we recommend to use the UI.

In [ ]:
# Login to Keycloak
CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)
response.raise_for_status()
access_token = response.json()['access_token']

# Lets inspect the token we got to see that our application name is available:
JSON(jwt.decode(access_token, options={"verify_signature": False}))

Now that we have the access token, we can query the server info Endpoint. 
On first launch it will show bootstrapped `'bootstrapped': false`.
The full API documentation is available as part of the Repository and hosted by Lakekeeper: http://localhost:8181/swagger-ui/#/

In [ ]:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())
# On first launch it shows "bootstrapped": False

## 2. Bootstrap

In [5]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/bootstrap",
    headers={
        "Authorization": f"Bearer {access_token}"
    },
    json={
        "accept-terms-of-use": True,
    },
)
response.raise_for_status()

## 3. Grant access to UI User
In keycloak the user "Peter" exists which we are now also assigning the "admin" role.

Before executing the next cell, login to the UI at http://localhost:8181 using:
* Username: `peter`
* Password: `iceberg`

You should see "You don't have any projects assignments".

Lets assign permissions to peter:

In [ ]:
# Users will show up in the /v1/user endpoint after the first login via the UI
# or the first call to the /catalog/v1/config endpoint.
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [7]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={
      "writes": [
        {
          "type": "admin",
          "user": "oidc~cfb55bf6-fcbb-4a1e-bfec-30c6649b52f8"
        }
      ]
    }
)
response.raise_for_status()

response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/permissions/project/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={
      "writes": [
        {
          "type": "project_admin",
          "user": "oidc~cfb55bf6-fcbb-4a1e-bfec-30c6649b52f8"
        }
      ]
    }
)
response.raise_for_status()

You can now refresh the UI page and should see the default Lakehouse.

In [ ]:
# The server is now bootstrapped:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
# An initial user was created
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
# The "admin" role has been assigned:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
user_id = response.json()['assignments'][0]['user']
JSON(response.json())

In [ ]:
# This user is the global admin, which has all access rights to the server:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/access",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
# Lets see who this user is:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user/{user_id}",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())